# Azure AI Agent service - Many agents

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
import json
import os
import sys
import time

from azure.ai.agents import AgentsClient
from azure.ai.agents.models import ListSortOrder
from azure.identity import DefaultAzureCredential
from datetime import datetime, timezone, timedelta
from dotenv import load_dotenv
from openai import AzureOpenAI

In [2]:
load_dotenv()

True

In [3]:
sys.version

'3.13.4 (main, Jun  3 2025, 15:34:24) [Clang 17.0.0 (clang-1700.0.13.3)]'

## Create connection to AI Foundry Project & instantiate a project client

In [4]:
endpoint = os.getenv("PROJECT_ENDPOINT")
print(f"Using endpoint: {endpoint}")
credential = DefaultAzureCredential()

project_client = AgentsClient(endpoint=endpoint, credential=credential)

Using endpoint: https://aq-ai-foundry-sweden-central.services.ai.azure.com/api/projects/firstProject


## Instantiate AGENT 1 (Q&A)

In [5]:


agent1 = project_client.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="Q&A",
    instructions="You are an AI agent.",
)
print(f"Created agent, agent ID: {agent1.id}")

Created agent, agent ID: asst_ctx1Y9bOXV0cesCV6Rdbj1dx


## Instantiate AGENT 2 (Translator)

In [6]:


agent2 = project_client.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="Translator",
    instructions="You are an AI agent that translates English into Spanish.",
)
print(f"Created agent, agent ID: {agent2.id}")

Created agent, agent ID: asst_1SRXyOL7LBqSDsJgagvZpfzz


## Instantiate AGENT 3 (Emojis creator)

In [7]:


agent3 = project_client.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="Emojis",
    instructions="You are an AI agent that add emojis to a text",
)
print(f"Created agent, agent ID: {agent3.id}")


Created agent, agent ID: asst_JqeB5OMrHVC1PJwzj3vbwmNP


### Show all agents

In [8]:
print(agent1)
print(agent2)
print(agent3)


{'id': 'asst_ctx1Y9bOXV0cesCV6Rdbj1dx', 'object': 'assistant', 'created_at': 1749846143, 'name': 'Q&A', 'description': None, 'model': 'gpt-4.1', 'instructions': 'You are an AI agent.', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}
{'id': 'asst_1SRXyOL7LBqSDsJgagvZpfzz', 'object': 'assistant', 'created_at': 1749846144, 'name': 'Translator', 'description': None, 'model': 'gpt-4.1', 'instructions': 'You are an AI agent that translates English into Spanish.', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}
{'id': 'asst_JqeB5OMrHVC1PJwzj3vbwmNP', 'object': 'assistant', 'created_at': 1749846144, 'name': 'Emojis', 'description': None, 'model': 'gpt-4.1', 'instructions': 'You are an AI agent that add emojis to a text', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}


### Create a thread for AGENT 1

In [ ]:
# [START create_thread]
thread = project_client.threads.create()
# [END create_thread]
print(f"Created thread, thread ID: {thread.id}")

Created thread, thread ID: thread_UTcicJXgO5jeQrA6bl7DNFnD


### Create a message for AGENT 1

In [ ]:

# [START create_message]
message = project_client.messages.create(thread_id=thread.id, role="user", content="Tell me about the 'Palacio de Bellas Artes' in Mexico City.")
# [END create_message]
print(f"Created message, message ID: {message.id}")

Created message, message ID: msg_S4yS1TAlN6DfTAzzgGeOpZTt


### Run AGENT 1

In [ ]:
# [START create_run]
run = project_client.runs.create(thread_id=thread.id, agent_id=agent1.id)
# [END create_run]
print(f"Created run, run ID: {run.id}")


Created run, run ID: run_1KdbKiJHPwoNRzJokmBJk1CW


### Check status of run for AGENT 1

In [ ]:
# Poll the run as long as run status is queued or in progress
while run.status in ["queued", "in_progress", "requires_action"]:
    # Wait for a second
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    # [END create_run]
    print(f"Run status: {run.status}")

    if run.status == "failed":
        print(f"Run error: {run.last_error}")

Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.COMPLETED


### Show usage (tokens)

In [13]:
run.usage

{'prompt_tokens': 41, 'completion_tokens': 448, 'total_tokens': 489, 'prompt_token_details': {'cached_tokens': 0}}

### Show me the stack of messages so far

In [14]:
messages = project_client.messages.list(thread_id=thread.id)
for message in messages:
    print(f"Message ID: {message.id}, Role: {message.role}, Content: {message.content}")





Message ID: msg_Zc62OmLFgna2TKbCwkSOVeCT, Role: MessageRole.AGENT, Content: [{'type': 'text', 'text': {'value': 'The **Palacio de Bellas Artes** (Palace of Fine Arts) is one of the most iconic cultural landmarks in **Mexico City**. Located in the historic center, near Alameda Central park, it serves as the national opera house and the main venue for the **Ballet Folklórico de México**, as well as numerous concerts, operas, and art exhibitions.\n\n**Key facts and features:**\n\n- **Construction and Architecture:**  \n  Construction began in 1904 under architect Adamo Boari but was delayed by the Mexican Revolution and technical challenges. It was completed in 1934 by Mexican architect Federico Mariscal.  \n  The palace exhibits a striking blend of **Art Nouveau** (exterior) and **Art Deco** (interior) styles, featuring white Carrara marble façade and opulent, decorative details.\n\n- **Murals and Art:**  \n  The interior is adorned with murals by renowned Mexican artists including **Die

### Show last message of AGENT 1

In [15]:
messages = list(project_client.messages.list(thread_id=thread.id))
if messages:
    last_message = messages[0]
    print(f"Content: {last_message.content}")
else:
    print("No messages found.")

Content: [{'type': 'text', 'text': {'value': 'The **Palacio de Bellas Artes** (Palace of Fine Arts) is one of the most iconic cultural landmarks in **Mexico City**. Located in the historic center, near Alameda Central park, it serves as the national opera house and the main venue for the **Ballet Folklórico de México**, as well as numerous concerts, operas, and art exhibitions.\n\n**Key facts and features:**\n\n- **Construction and Architecture:**  \n  Construction began in 1904 under architect Adamo Boari but was delayed by the Mexican Revolution and technical challenges. It was completed in 1934 by Mexican architect Federico Mariscal.  \n  The palace exhibits a striking blend of **Art Nouveau** (exterior) and **Art Deco** (interior) styles, featuring white Carrara marble façade and opulent, decorative details.\n\n- **Murals and Art:**  \n  The interior is adorned with murals by renowned Mexican artists including **Diego Rivera**, **David Alfaro Siqueiros**, **Rufino Tamayo**, and **J

### SHow bottom of stack of messages

In [16]:
if messages:
    last_message = messages[-1]
    print(f"Content: {last_message.content}")
else:
    print("No messages found.")

Content: [{'type': 'text', 'text': {'value': "Tell me about the 'Palacio de Bellas Artes' in Mexico City.", 'annotations': []}}]


### Run AGENT 2

In [ ]:
# [START create_run]
run = project_client.runs.create(thread_id=thread.id, agent_id=agent2.id)
# [END create_run]
print(f"Created run, run ID: {run.id}")

Created run, run ID: run_xYXKHLgq60bfUcImUURXZGKV


### Check Status of the run for AGENT 2

In [ ]:
# Poll the run as long as run status is queued or in progress
 while run.status in ["queued", "in_progress", "requires_action"]:
    # Wait for a second
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    # [END create_run]
    print(f"Run status: {run.status}")

    if run.status == "failed":
        print(f"Run error: {run.last_error}")

Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.COMPLETED


In [19]:
run.usage

{'prompt_tokens': 496, 'completion_tokens': 300, 'total_tokens': 796, 'prompt_token_details': {'cached_tokens': 0}}

### SHow the stack of messages so far

In [20]:
messages = project_client.messages.list(thread_id=thread.id)
for message in messages:
    print(f"Message ID: {message.id}, Role: {message.role}, Content: {message.content}")



Message ID: msg_HUj59w2V1wuyCTwxj89FAPcN, Role: MessageRole.AGENT, Content: [{'type': 'text', 'text': {'value': 'El **Palacio de Bellas Artes** es uno de los recintos culturales más emblemáticos de la Ciudad de México. Ubicado en el centro histórico, junto a la Alameda Central, es el principal escenario para la ópera, la danza y diversas expresiones artísticas del país.\n\nSu construcción comenzó en 1904 bajo la dirección del arquitecto Adamo Boari, aunque se concluyó en 1934 por Federico Mariscal, debido a diversos retrasos, incluyendo la Revolución Mexicana. El edificio combina estilos arquitectónicos Art Nouveau en el exterior y Art Decó en su interior, y está revestido de mármol blanco de Carrara.\n\nDentro del Palacio se encuentran murales de grandes artistas mexicanos como Diego Rivera, David Alfaro Siqueiros, Rufino Tamayo y José Clemente Orozco. Una de sus piezas más singulares es el telón de cristal creado por Tiffany & Co., que representa los volcanes Popocatépetl e Iztaccíhu

### SHow last message of AGENT 2

In [21]:
messages = list(project_client.messages.list(thread_id=thread.id))
if messages:
    last_message = messages[0]
    print(f"Content: {last_message.content}")
else:
    print("No messages found.")

Content: [{'type': 'text', 'text': {'value': 'El **Palacio de Bellas Artes** es uno de los recintos culturales más emblemáticos de la Ciudad de México. Ubicado en el centro histórico, junto a la Alameda Central, es el principal escenario para la ópera, la danza y diversas expresiones artísticas del país.\n\nSu construcción comenzó en 1904 bajo la dirección del arquitecto Adamo Boari, aunque se concluyó en 1934 por Federico Mariscal, debido a diversos retrasos, incluyendo la Revolución Mexicana. El edificio combina estilos arquitectónicos Art Nouveau en el exterior y Art Decó en su interior, y está revestido de mármol blanco de Carrara.\n\nDentro del Palacio se encuentran murales de grandes artistas mexicanos como Diego Rivera, David Alfaro Siqueiros, Rufino Tamayo y José Clemente Orozco. Una de sus piezas más singulares es el telón de cristal creado por Tiffany & Co., que representa los volcanes Popocatépetl e Iztaccíhuatl.\n\nAdemás de funcionar como teatro y sala de conciertos, el Pa

### Run AGENT 3

In [ ]:
# [START create_run]
run = project_client.runs.create(thread_id=thread.id, agent_id=agent3.id)
 # [END create_run]
print(f"Created run, run ID: {run.id}")

Created run, run ID: run_K85FAxUuFrr64fvMZWjXPfh4


### Check status of run of AGENT 3

In [ ]:
# Poll the run as long as run status is queued or in progress
while run.status in ["queued", "in_progress", "requires_action"]:
    # Wait for a second
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    # [END create_run]
    print(f"Run status: {run.status}")

    if run.status == "failed":
        print(f"Run error: {run.last_error}")

Run status: RunStatus.IN_PROGRESS
Run status: RunStatus.COMPLETED


In [24]:
run.usage

{'prompt_tokens': 798, 'completion_tokens': 224, 'total_tokens': 1022, 'prompt_token_details': {'cached_tokens': 0}}

### Show the Stack of messages so far

In [25]:
messages = project_client.messages.list(thread_id=thread.id)
for message in messages:
    print(f"Message ID: {message.id}, Role: {message.role}, Content: {message.content}")

Message ID: msg_5kQ05UXkSKKmDj7IZPMWvHxY, Role: MessageRole.AGENT, Content: [{'type': 'text', 'text': {'value': 'Absolutely! Here’s your answer with emojis:\n\nThe **Palacio de Bellas Artes** 🎭🏛️ is one of Mexico City’s 🌆 most iconic landmarks. Located in the historic center next to Alameda Central 🌳, it’s famous for its stunning architecture, which blends Art Nouveau on the outside 🏰 with Art Deco on the inside 🪞.\n\nConstruction started in 1904 and was completed in 1934. The palace is home to some of Mexico’s most important artistic events, such as operas 🎶, concerts 🎻, and performances by the Ballet Folklórico de México 💃.\n\nInside, you’ll find breathtaking murals 🎨 by celebrated artists like Diego Rivera and Rufino Tamayo. Another highlight is its amazing Tiffany glass stage curtain 🖼️ depicting the Valley of Mexico with its volcanoes 🌋.\n\nThe Palacio de Bellas Artes also houses important museums 🖼️ and is recognized as a UNESCO World Heritage Site 🌍. It’s truly a must-visit for 

### Print last message of AGENT 3, this shows all Agents work combined in final output!!

In [26]:
messages = list(project_client.messages.list(thread_id=thread.id))

last_message = messages[0]
print(f"Content: {last_message.content}")




Content: [{'type': 'text', 'text': {'value': 'Absolutely! Here’s your answer with emojis:\n\nThe **Palacio de Bellas Artes** 🎭🏛️ is one of Mexico City’s 🌆 most iconic landmarks. Located in the historic center next to Alameda Central 🌳, it’s famous for its stunning architecture, which blends Art Nouveau on the outside 🏰 with Art Deco on the inside 🪞.\n\nConstruction started in 1904 and was completed in 1934. The palace is home to some of Mexico’s most important artistic events, such as operas 🎶, concerts 🎻, and performances by the Ballet Folklórico de México 💃.\n\nInside, you’ll find breathtaking murals 🎨 by celebrated artists like Diego Rivera and Rufino Tamayo. Another highlight is its amazing Tiffany glass stage curtain 🖼️ depicting the Valley of Mexico with its volcanoes 🌋.\n\nThe Palacio de Bellas Artes also houses important museums 🖼️ and is recognized as a UNESCO World Heritage Site 🌍. It’s truly a must-visit for anyone interested in art, architecture, and Mexican culture! 🇲🇽', '

## Post processing

In [27]:
# List all agents in the project
print("Listing all agents in the project:")
agents = project_client.list_agents()
for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, Instructions: {agent.instructions}")

Listing all agents in the project:
Agent ID: asst_JqeB5OMrHVC1PJwzj3vbwmNP, Name: Emojis, Model: gpt-4.1, Instructions: You are an AI agent that add emojis to a text
Agent ID: asst_1SRXyOL7LBqSDsJgagvZpfzz, Name: Translator, Model: gpt-4.1, Instructions: You are an AI agent that translates English into Spanish.
Agent ID: asst_ctx1Y9bOXV0cesCV6Rdbj1dx, Name: Q&A, Model: gpt-4.1, Instructions: You are an AI agent.


In [29]:
# recurse through all agents and delete them adding a test to stop if the agent is not found
for agent in project_client.list_agents():
    try:
        print(f"Deleting agent ID: {agent.id}, Name: {agent.name}")
        project_client.delete_agent(agent.id)
    except Exception as e:
        print(f"Error deleting agent ID: {agent.id}, Name: {agent.name}, Error: {e}")
        break

In [30]:
agents = project_client.list_agents()
for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, Instructions: {agent.instructions}")